In [37]:
import boa
import sys, os
from eth_utils import to_checksum_address
parent_dir = os.path.abspath('..')
sys.path.insert(0, parent_dir)
from tests.utils.util_tokens import mint_for_testing, get_token_balance
from tests.utils.util_contracts import load_contract


from dotenv import load_dotenv
dotenv_path = os.path.expanduser('~/.web3env') # Path to .env file containing rpcs and keys
load_dotenv(dotenv_path)

rpc_url = os.getenv("ETH_RPC_URL")
assert rpc_url is not None, "Provider url is not set"
boa.env.fork(url=rpc_url, block_identifier=20420069)
boa.env.enable_fast_mode()

addresses_dict = {
    "USDT":     '0xdAC17F958D2ee523a2206206994597C13D831ec7',
    "USDC":     '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
    # "DAI":      '0x6b175474e89094c44da98b954eedeac495271d0f', #curve doesn't like DAI it seems
    "WBTC":     '0x2260fac5e5542a773aa44fbcfedf7c193bc2c599',
    "WETH":     '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
    "CRV":      '0xD533a949740bb3306d119CC777fa900bA034cd52',
    "crvUSD":   '0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E'
}
# Convert addresses to checksum format
for key in addresses_dict.keys():
    addresses_dict[key] = to_checksum_address(addresses_dict[key])
# Fetch contract data from Etherscan and create the contracts dictionary
# contracts = {key: boa.from_etherscan(address, uri="https://api.etherscan.io/api", name=key, api_key=os.environ.get("ETHERSCAN_API_KEY_CS")) for key, address in addresses_dict.items()}
token_contracts = {key: load_contract(address, name=key, path_prefix='../') for key, address in addresses_dict.items()}

print(f'Forked the chain on block {boa.env.evm.vm.state.block_number}')
boa.env.set_random_seed(420)
deployer = boa.env.generate_address()
with boa.env.prank(deployer):
    curve_assistant_contract = boa.load_partial("../contracts/CurveAssistant.vy").deploy()

# load prices for value estimation
BTC_PRICE = load_contract('0x7f86bf177dd4f3494b841a37e810a34dd56c829b', name=key, path_prefix='../').last_prices(0)/1e18
ETH_PRICE = load_contract('0x7f86bf177dd4f3494b841a37e810a34dd56c829b', name=key, path_prefix='../').last_prices(1)/1e18
CRV_PRICE = load_contract('0x4ebdf703948ddcea3b11f675b4d1fba9d2414a14', name=key, path_prefix='../').last_prices(1)/1e18
print(f'Prices: BTC: {BTC_PRICE:.2f}, ETH: {ETH_PRICE:.2f}, CRV: {CRV_PRICE:.3f}')


Forked the chain on block 20420069
Prices: BTC: 66057.12, ETH: 3308.77, CRV: 0.278


In [38]:
pools_list = ['0x7f86bf177dd4f3494b841a37e810a34dd56c829b', # tricrypto-ng USDC/WBTC/WETH
                '0xd51a44d3fae010294c616388b506acda1bfaae46', # Curve.fi: USDT/WBTC/WETH Pool (old)
                '0xf5f5b97624542d72a9e06f04804bf81baa15e2b4', # tricrypto-ng USDT/WBTC/WETH
                '0x4ebdf703948ddcea3b11f675b4d1fba9d2414a14', # tricrv (crvUSD/WETH/CRV)
                ] 
coins_list = []
coins_amounts = []

AMT_USD = 1 * 10 ** 4
AMT_ETH = 10 ** 1
AMT_BTC = 10 ** 0
AMT_OTHER = 10 ** 4

for key in token_contracts:
    contract = token_contracts[key]
    with boa.env.prank(deployer):
        coins_list.append(contract.address)
        AMT_TO_MINT = 0
        if 'usd' in key.lower():
            AMT_TO_MINT = AMT_USD * 10 ** token_contracts[key].decimals()
        elif 'eth' in key.lower():
            AMT_TO_MINT = AMT_ETH * 10 ** token_contracts[key].decimals()
        elif 'btc' in key.lower():
            AMT_TO_MINT = AMT_BTC * 10 ** token_contracts[key].decimals()
        else:
            AMT_TO_MINT = AMT_OTHER * 10 ** token_contracts[key].decimals()
        mint_for_testing(contract, deployer, AMT_TO_MINT, path_prefix='../')
        coins_amounts.append(contract.balanceOf(deployer) // 2)
        print(f'{contract.name()} balance: {contract.balanceOf(deployer)/10 ** contract.decimals()}')
        tx = contract.approve(curve_assistant_contract.address, 0, sender = deployer)
        tx = contract.approve(curve_assistant_contract.address, contract.balanceOf(deployer), sender = deployer)
        # print(f'{contract.name()} allowance: {contract.allowance(deployer, curve_assistant_contract.address)/10 ** contract.decimals()}')
print('Tokens minted to deployer and approved for CurveAssistant contract')

Tether USD balance: 10000.0
USD Coin balance: 10000.0
Wrapped BTC balance: 1.0
Wrapped Ether balance: 10.0
Curve DAO Token balance: 10000.0
Curve.Fi USD Stablecoin balance: 10000.0
Tokens minted to deployer and approved for CurveAssistant contract


In [39]:
target_coin = addresses_dict['USDT']
target_coin = to_checksum_address(target_coin)
coins_amounts = []
convert_list = []
value_init = 0
target_coin_contract = load_contract(target_coin, path_prefix='../')
for coin in coins_list: 
    coin_contract = load_contract(coin, path_prefix='../')
    balance = coin_contract.balanceOf(deployer)
    contract_name = coin_contract.name()
    tmp_value = 0   
    if 'usd' in contract_name.lower():
        tmp_value = (balance / 10 ** coin_contract.decimals())
    elif 'eth' in contract_name.lower():
        tmp_value = ETH_PRICE * (balance / 10 ** coin_contract.decimals())
    elif 'btc' in contract_name.lower():
        #value_prev += 60500 * (balance / 10 ** coin_contract.decimals())
        tmp_value = BTC_PRICE * (balance / 10 ** coin_contract.decimals())
    elif 'curve' in contract_name.lower():
        #value_prev += 0.25 * (balance / 10 ** coin_contract.decimals())
        tmp_value = CRV_PRICE * (balance / 10 ** coin_contract.decimals())
    value_init += tmp_value
    print(f'Balance of {contract_name} ({coin_contract.address}):', balance/10**coin_contract.decimals() , f'({tmp_value:.2f} USD)')

    if balance > 0 and coin != target_coin:
        convert_list.append(coin)
        coins_amounts.append(balance)
        with boa.env.prank(deployer):
            coin_contract.approve(curve_assistant_contract.address, 0, sender = deployer)
            coin_contract.approve(curve_assistant_contract.address, balance, sender = deployer)
print(f'Initial value: {value_init:.2f} USD')
with boa.env.prank(deployer):
    # contract_value = curve_assistant_contract.estimate_usd_value_of_coins(convert_list, coins_amounts, pools_list, sender=deployer)
    commit_value = curve_assistant_contract.commit_slippage_protection(convert_list, coins_amounts, pools_list, target_coin, 10_0, sender=deployer)
    print(f'commit_value: {commit_value}')
    res_convert = curve_assistant_contract.convert_to_one(convert_list, coins_amounts, pools_list, target_coin, 1, sender=deployer)
    print(f'received tokens: {res_convert}')

print(f'Swapping {convert_list} to {target_coin}')
print(f'Balance of {target_coin_contract.name()} after conversion:', f'{get_token_balance(target_coin, deployer)/10**target_coin_contract.decimals():.2f}')


Balance of Tether USD (0xdAC17F958D2ee523a2206206994597C13D831ec7): 10000.0 (10000.00 USD)
Balance of USD Coin (0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48): 10000.0 (10000.00 USD)
Balance of Wrapped BTC (0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599): 1.0 (66057.12 USD)
Balance of Wrapped Ether (0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2): 10.0 (33087.69 USD)
Balance of Curve DAO Token (0xD533a949740bb3306d119CC777fa900bA034cd52): 10000.0 (2783.18 USD)
Balance of Curve.Fi USD Stablecoin (0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E): 10000.0 (10000.00 USD)
Initial value: 131927.99 USD
commit_value: 110226420891
returned value: 120932956836
Swapping [Address('0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'), Address('0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599'), Address('0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'), Address('0xD533a949740bb3306d119CC777fa900bA034cd52'), Address('0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E')] to 0xdAC17F958D2ee523a2206206994597C13D831ec7
Balance of Tether USD afte

In [40]:
# iterations benchmark for all -> usdt (10k stables each, 10e, 1btc, 10k crv balances initially)
# 1 iter: 131927.99 -> 130932.96 (-0.754%)
# 2 iter: 131927.99 -> 131011.57 (-0.694%)
# 3 iter: 131927.99 -> 131029.44 (-0.681%)
# 5 iter: 131927.99 -> 131063.44 (-0.656%)
# 7 iter: 131927.99 -> 131066.22 (-0.654%)
# 9 iter: 131927.99 -> 131074.00 (-0.648%)

# iterations benchmark for all -> usdt (100k stables each, 100e, 10btc, 100k crv balances initially)
# 1 iter: 1319279.91 -> 1243184.22 (-5.764%)
# 2 iter: 1319279.91 -> 1248796.39 (-5.347%)
# 3 iter: 1319279.91 -> 1248982.88 (-5.325%)
# 5 iter: 1319279.91 -> 1249971.26 (-5.261%)
# 7 iter: 1319279.91 -> 1250552.76 (-5.218%)
# 9 iter: 1319279.91 -> 1250991.21 (-5.177%)

# now try to do stables -> crvUSD (50k stables each, 0e, 0btc, 0crv balances initially)
# 1 iter: 100000.00 -> 97252.08 (-2.748%)
# 9 iter: 100000.00 -> 97494.09 (-2.506%)

# large bag stables -> crvUSD (500k stables each, 0e, 0btc, 0crv balances initially)
# 1 iter: 1000000.00 -> 835310.45 (-16.469%)
# 9 iter: 1000000.00 -> 841730.84 (-15.827%)